# Train llama3 with LoRA
[Source of this notebook](https://medium.com/@avishekpaul31/fine-tuning-llama-3-8b-instruct-qlora-using-low-cost-resources-89075e0dfa04)

This notebook is designed for Kaggle notebook with 2 Nvidia T4 GPUs

### Enviornment setup
- Set your `HF_TOKEN` at `Add-ons -> Secrets`

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
# HF_TOKEN_WRITE = user_secrets.get_secret("HF_TOKEN_WRITE")
# HF_TOKEN_WRITE = user_secrets.get_secret("HF_TOKEN_WRITE")

In [2]:
!pip install huggingface_hub

# from huggingface_hub import notebook_login
# notebook_login()

!huggingface-cli login --token $HF_TOKEN
!pip install -U transformers[torch] datasets
!pip install -q bitsandbytes trl peft accelerate
!pip install flash-attn --no-build-isolation














Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.

Token is valid (permission: write).

Your token has been saved to /root/.cache/huggingface/token

Login successful







     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 1.6 MB/s eta 0:00:00a 0:00:01













































   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.7 MB/s eta 0:00:00:00:010:01


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 35.3 MB/s eta 0:00:00a 0:00:01


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 68.3 MB/s eta 0:00:00:00:0100:01


  Attempting uninstall: tokenizers

    Found existing installation: tokenizers 0.15.2

    Uninstalling tokenizers-0.15.2:

      Successfully uninstalled tokenizers-0.15.2

  Attempting uninstall: transformers

    Found existing installation: transformers 4.39.3

    Uninstalling transformers-4.39.3:

      Successfully un

### Load dataset

In [5]:
from datasets import load_dataset, DatasetDict

# based on config
raw_datasets = load_dataset("HuggingFaceH4/ultrachat_200k")

# remove this when done debugging to include whole dataset
indices = range(0,500)

dataset_dict = {"train": raw_datasets["train_sft"].select(indices),
                "test": raw_datasets["test_sft"].select(indices)}

raw_datasets = DatasetDict(dataset_dict)
raw_datasets

Generating train_sft split:   0%|          | 0/207865 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/23110 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/256032 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/28304 [00:00<?, ? examples/s]

In [7]:
raw_datasets["train"][:2]['messages']

[[{'content': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?",
   'role': 'user'},
  {'content': 'This feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.',
   'role': 'assistant'},
  {'content': 'Can you guide me through the process of enabling the secondary image hover feature on my Collection pages and Featured Collections sections?',
   'role': 'user'},
  {'content': "Sure, h

In [8]:
example = raw_datasets["train"][0]
messages = example["messages"]
for message in messages:
  role = message["role"]
  content = message["content"]
  print('{0:20}:  {1}'.format(role, content))

user                :  These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?

On your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!

Your Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.

Does this feature apply to all sections of the theme or just specific ones as listed in the text material?

assistant           :  This feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.

user                :  Can you guide me through the process of enabling the secondary image hover feature on my Collection pages and Featured Collections sections?

assistant           :  Sure, here are the steps to enable the seco

### Load and setup base model

In [10]:
from transformers import AutoTokenizer
from accelerate import PartialState

# model_id = "mistralai/Mistral-7B-v0.2"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, device_map="auto")
tokenizer.eos_token_id
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
import re
import random
from multiprocessing import cpu_count

def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)

    return example

In [14]:
column_names = list(raw_datasets["train"].features)
raw_datasets = raw_datasets.map(apply_chat_template,
                                num_proc=cpu_count(),
                                fn_kwargs={"tokenizer": tokenizer},
                                remove_columns=column_names,
                                desc="Applying chat template",)

raw_datasets

Applying chat template (num_proc=4):   0%|          | 0/500 [00:00<?, ? examples/s]

Applying chat template (num_proc=4):   0%|          | 0/500 [00:00<?, ? examples/s]

In [15]:
# set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
  tokenizer.model_max_length = 2048

In [16]:
# create the splits
train_dataset = raw_datasets["train"]
eval_dataset = raw_datasets["test"]

for index in random.sample(range(len(raw_datasets["train"])), 3):
  print(f"Sample {index} of the processed training set:\n\n{raw_datasets['train'][index]['text']}")
  print("#####################################")

Sample 306 of the processed training set:



<|begin_of_text|><|start_header_id|>system<|end_header_id|>



<|eot_id|><|start_header_id|>user<|end_header_id|>



Given the text: Applications are invited for an NIH funded postdoctoral position combining eye tracking and intracranial EEG recordings in humans to study the neural basis of face recognition, object recognition, and social and affective perception, at the University of Pittsburgh, USA.

The research will focus on understanding the dynamic neural code that underlies the recognition of faces, bodies, objects, facial expressions, and other social and affective stimuli and how this information guides, and is guided by, eye movements. Of particular interest is how this information is coded in interactive neural circuits at the level of large-scale brain networks. The neural data will primarily be local field potentials/ event related potential from intracranial surface electrodes (electrocorticography, ECoG), cortical depth electr

### Training

In [17]:
from transformers import BitsAndBytesConfig
import torch

# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
)

# Set device_map to "auto" to allow multi-GPU training
device_map = "auto"#{"": torch.cuda.current_device()} if torch.cuda.is_available() else None

model_kwargs = dict(
#     attn_implementation=False,#"flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
    torch_dtype="auto",
    use_cache=False, # set to False as we're going to use gradient checkpointing
    device_map=device_map,
    quantization_config=quantization_config,
)

In [18]:
from trl import SFTTrainer
from peft import LoraConfig
from transformers import TrainingArguments

2024-05-02 01:37:18.741914: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered

2024-05-02 01:37:18.742013: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered

2024-05-02 01:37:18.911854: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [19]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [20]:
# path where the Trainer will save its checkpoints and logs
trained_model_id = "Llama-3-8B-sft-lora-ultrachat"
output_dir = 'kaggle/working/' + trained_model_id

# based on config
training_args = TrainingArguments(
    fp16=False, # specify bf16=True instead when training on GPUs that support bf16 else fp16
    bf16=False,
    do_eval=True,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    learning_rate=2.0e-05,
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_steps=-1,
    num_train_epochs=1,
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_eval_batch_size=1, # originally set to 8
    per_device_train_batch_size=1, # originally set to 8
    push_to_hub=False,
    hub_model_id=trained_model_id,
    # hub_strategy="every_save",
    # report_to="tensorboard",
    report_to="none",
    save_strategy="no",
    save_total_limit=None,
    seed=42,
)

In [21]:
# based on config
peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

In [22]:
# del trainer
torch.cuda.empty_cache()

In [23]:
trainer = SFTTrainer(
        model=model_id,
        model_init_kwargs=model_kwargs,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        dataset_text_field="text",
        tokenizer=tokenizer,
        packing=True,
        peft_config=peft_config,
    )

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:166: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.

  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024

  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2991 > 2048). Running this sequence through the model will result in indexing errors


Generating train split: 0 examples [00:00, ? examples/s]

You have loaded a model on multiple GPUs. `is_model_parallel` attribute will be force-set to `True` to avoid any unexpected behavior such as device placement mismatching.


In [ ]:
!export CUDA_VISIBLE_DEVICES=0,1

In [27]:
train_result = trainer.train()

***** Running training *****

  Num examples = 556

  Num Epochs = 1

  Instantaneous batch size per device = 1

  Total train batch size (w. parallel, distributed & accumulation) = 1

  Gradient Accumulation steps = 1

  Total optimization steps = 556

  Number of trainable parameters = 54,525,952


Epoch,Training Loss,Validation Loss
1,1.222300,1.362127


***** Running Evaluation *****

  Num examples = 574

  Batch size = 1





Training completed. Do not forget to share your model on huggingface.co/models =)






### Save the trained model

In [34]:
print('Finished')
trainer.save_model(training_args.output_dir)

Saving model checkpoint to kaggle/working/Llama-3-8B-sft-lora-ultrachat


Finished


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/e5e23bbe8e749ef0efcf16cad411a7d23bd23298/config.json

Model config LlamaConfig {

  "architectures": [

    "LlamaForCausalLM"

  ],

  "attention_bias": false,

  "attention_dropout": 0.0,

  "bos_token_id": 128000,

  "eos_token_id": 128001,

  "hidden_act": "silu",

  "hidden_size": 4096,

  "initializer_range": 0.02,

  "intermediate_size": 14336,

  "max_position_embeddings": 8192,

  "model_type": "llama",

  "num_attention_heads": 32,

  "num_hidden_layers": 32,

  "num_key_value_heads": 8,

  "pretraining_tp": 1,

  "rms_norm_eps": 1e-05,

  "rope_scaling": null,

  "rope_theta": 500000.0,

  "tie_word_embeddings": false,

  "torch_dtype": "bfloat16",

  "transformers_version": "4.40.1",

  "use_cache": true,

  "vocab_size": 128256

}



tokenizer config file saved in kaggle/working/Llama-3-8B-sft-lora-ultrachat/tokenizer_config.json

Special

adapter_model.safetensors:   0%|          | 0.00/218M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

## Useful sources
- https://www.databricks.com/blog/efficient-fine-tuning-lora-guide-llms